In [1]:
import pandas as pd

# URL do dataset Wine
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data'

# Nome das colunas em português BR
column_names_pt_br = [
    'classe',
    'alcool',
    'acido_malico',
    'cinzas',
    'alcalinidade_de_cinzas',
    'magnesio',
    'fenois_totais',
    'flavanoides',
    'fenois_nao_flavanoides',
    'proantocianinas',
    'intensidade_de_cor',
    'matiz',
    'od280_od315_de_vinhos_diluidos',
    'prolina'
]

# Ler o arquivo CSV com as colunas especificadas, definindo a coluna 'classe' como object
vinhos = pd.read_csv(url, names=column_names_pt_br, dtype={'classe': object})

In [2]:
# Separar as features (X) e a target (y)
X = vinhos.iloc[:, 1:]
y = vinhos['classe']

# Verificando as primeiras linhas de X e y
print("Primeiras linhas de X:")
print(X.head())
print("\nPrimeiras linhas de y:")
print(y.head())


Primeiras linhas de X:
   alcool  acido_malico  cinzas  alcalinidade_de_cinzas  magnesio  \
0   14.23          1.71    2.43                    15.6       127   
1   13.20          1.78    2.14                    11.2       100   
2   13.16          2.36    2.67                    18.6       101   
3   14.37          1.95    2.50                    16.8       113   
4   13.24          2.59    2.87                    21.0       118   

   fenois_totais  flavanoides  fenois_nao_flavanoides  proantocianinas  \
0           2.80         3.06                    0.28             2.29   
1           2.65         2.76                    0.26             1.28   
2           2.80         3.24                    0.30             2.81   
3           3.85         3.49                    0.24             2.18   
4           2.80         2.69                    0.39             1.82   

   intensidade_de_cor  matiz  od280_od315_de_vinhos_diluidos  prolina  
0                5.64   1.04                 

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Lista de random_states para os experimentos
random_states = [42, 17, 24]

# Dicionário para armazenar os resultados
results = {}

for state in random_states:
    print(f"\nExperimentando com random_state = {state}")

    # Treino e teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=state)

    # KNN com k=3
    knn_k3 = KNeighborsClassifier(n_neighbors=3)
    knn_k3.fit(X_train, y_train)
    y_pred_k3 = knn_k3.predict(X_test)
    accuracy_k3 = accuracy_score(y_test, y_pred_k3)
    print(f"Acurácia do KNN com k=3: {accuracy_k3:.2f}")

    # KNN com k=5
    knn_k5 = KNeighborsClassifier(n_neighbors=5)
    knn_k5.fit(X_train, y_train)
    y_pred_k5 = knn_k5.predict(X_test)
    accuracy_k5 = accuracy_score(y_test, y_pred_k5)
    print(f"Acurácia do KNN com k=5: {accuracy_k5:.2f}")

    # Armazenar os resultados
    results[state] = {
        "k=3": accuracy_k3,
        "k=5": accuracy_k5
    }

# Resultados
print("\n### Resumo Final dos Resultados ###")
for state, accuracies in results.items():
    print(f"Random State: {state}, Acurácia k=3: {accuracies['k=3']:.2f}, Acurácia k=5: {accuracies['k=5']:.2f}")



Experimentando com random_state = 42
Acurácia do KNN com k=3: 0.74
Acurácia do KNN com k=5: 0.74

Experimentando com random_state = 17
Acurácia do KNN com k=3: 0.65
Acurácia do KNN com k=5: 0.67

Experimentando com random_state = 24
Acurácia do KNN com k=3: 0.74
Acurácia do KNN com k=5: 0.67

### Resumo Final dos Resultados ###
Random State: 42, Acurácia k=3: 0.74, Acurácia k=5: 0.74
Random State: 17, Acurácia k=3: 0.65, Acurácia k=5: 0.67
Random State: 24, Acurácia k=3: 0.74, Acurácia k=5: 0.67


In [4]:
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import numpy as np

# Função para realizar o KFold Cross-Validation
def kfold_cross_validation(X, y, random_state, k_values=[3, 5]):
    # KFold com 10 splits
    kf = KFold(n_splits=10, shuffle=True, random_state=random_state)

    # Armazenar as acurácias para cada valor de k
    accuracies = {k: [] for k in k_values}

    # Iterar pelos folds
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Treinamento e avaliação para cada valor de k
        for k in k_values:
            knn = KNeighborsClassifier(n_neighbors=k)
            knn.fit(X_train, y_train)

            # Previsões e calcular a acurácia
            y_pred = knn.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            accuracies[k].append(accuracy)

    # Média e desvio padrão para cada k
    for k in k_values:
        mean_accuracy = np.mean(accuracies[k])
        std_accuracy = np.std(accuracies[k])
        print(f"Random State: {random_state}, k={k} -> Média: {mean_accuracy:.2f}, Desvio Padrão: {std_accuracy:.2f}")
        return accuracies

# Experimentar com diferentes random_states
random_states = [42, 17, 24]
for state in random_states:
    print(f"\n### Cross-Validation para random_state = {state} ###")
    kfold_cross_validation(X, y, random_state=state)



### Cross-Validation para random_state = 42 ###
Random State: 42, k=3 -> Média: 0.70, Desvio Padrão: 0.14

### Cross-Validation para random_state = 17 ###
Random State: 17, k=3 -> Média: 0.70, Desvio Padrão: 0.10

### Cross-Validation para random_state = 24 ###
Random State: 24, k=3 -> Média: 0.71, Desvio Padrão: 0.10


In [5]:
import numpy as np

# KFold Cross-Validation
def kfold_cross_validation(X, y, random_state, k_values=[3, 5]):
    # Configuração do KFold com 10 splits
    kf = KFold(n_splits=10, shuffle=True, random_state=random_state)

    # Armazenar as acurácias para cada valor de k
    accuracies = {k: [] for k in k_values}

    # Itera
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Treinamento e avaliação para cada valor de k
        for k in k_values:
            knn = KNeighborsClassifier(n_neighbors=k)
            knn.fit(X_train, y_train)

            # Fazer previsões e calcular a acurácia
            y_pred = knn.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            accuracies[k].append(accuracy)

    # Calcular a média e desvio padrão para cada k
    for k in k_values:
        mean_accuracy = np.mean(accuracies[k])
        std_accuracy = np.std(accuracies[k])
        print(f"Random State: {random_state}, k={k} -> Média: {mean_accuracy:.2f}, Desvio Padrão: {std_accuracy:.2f}")

    return accuracies

# Experimentos com diferentes random_states
random_states = [42, 17, 24]
best_k_per_state = {}

for state in random_states:
    print(f"\n### Analisando para random_state = {state} ###")
    accuracies = kfold_cross_validation(X, y, random_state=state)

    # Comparando para k=3 e k=5
    mean_accuracy_k3 = np.mean(accuracies[3])
    mean_accuracy_k5 = np.mean(accuracies[5])

    # Identificando o melhor valor de k
    if mean_accuracy_k3 > mean_accuracy_k5:
        best_k_per_state[state] = 3
        print(f"Melhor k para random_state = {state}: k=3")
    elif mean_accuracy_k5 > mean_accuracy_k3:
        best_k_per_state[state] = 5
        print(f"Melhor k para random_state = {state}: k=5")
    else:
        best_k_per_state[state] = 'Empate'
        print(f"Houve empate para random_state = {state} entre k=3 e k=5")

# Resumo final dos melhores k por random_state
print("\n### Resumo dos melhores valores de k por random_state ###")
for state, best_k in best_k_per_state.items():
    print(f"Random State: {state}, Melhor k: {best_k}")



### Analisando para random_state = 42 ###
Random State: 42, k=3 -> Média: 0.70, Desvio Padrão: 0.14
Random State: 42, k=5 -> Média: 0.66, Desvio Padrão: 0.10
Melhor k para random_state = 42: k=3

### Analisando para random_state = 17 ###
Random State: 17, k=3 -> Média: 0.70, Desvio Padrão: 0.10
Random State: 17, k=5 -> Média: 0.69, Desvio Padrão: 0.11
Melhor k para random_state = 17: k=3

### Analisando para random_state = 24 ###
Random State: 24, k=3 -> Média: 0.71, Desvio Padrão: 0.10
Random State: 24, k=5 -> Média: 0.71, Desvio Padrão: 0.09
Houve empate para random_state = 24 entre k=3 e k=5

### Resumo dos melhores valores de k por random_state ###
Random State: 42, Melhor k: 3
Random State: 17, Melhor k: 3
Random State: 24, Melhor k: Empate


#Para o experimento com random_state = 42:
A diferença entre os dois foi pequena, mas ainda assim, k=3 apresentou o melhor desempenho, ainda que de forma bem equilibrada.

#Para o experimento com random_state = 24:
Para este valor de random_state, os modelos com 𝑘=3 e 𝑘=5
tiveram o mesmo desempenho, ou seja, houve um empate entre os dois valores de 𝑘

#Para o experimento com random_state = 17:
Aqui, o valor de 𝑘=3 teve uma média de 0.70, com um desvio padrão de 0.10. O modelo com 𝑘=5 obteve uma média de 0.69 e desvio padrão de 0.11.
A diferença entre os dois foi pequena, mas ainda assim, k=3 apresentou o melhor desempenho, ainda que de forma bem equilibrada.

k=3 foi a escolha mais vantajosa, embora o empate tenha ocorrido para k=3 e k=5 no caso do random_state = 24.